# Lets get this sorted

- https://adventofcode.com/2023/day/7

Today's challenge is to sort hands of cards. This is actually quite simple. Just use a [`Counter()`](https://docs.python.org/3/library/collections.html#collections.Counter) on the hand and only keep the ordered values (counts) of the hand. This will match each of the hand _types_ in the puzzle description, and they are ordered exactly as the puzzle requires:

- `(5)` is Five of a kind
- `(4, 1)` is Four of a kind
- `(3, 2)` is Full house
- `(3, 1, 1)` is Three of a kind
- `(2, 2, 1)` is Two pair
- `(2, 1, 1, 1)` is One pair
- `(1, 1, 1, 1, 1)` is High card

Reversed lexicographical sorting of Python tuples would put them in exactly that order.

To break ties between two hands with the same type, just compare the hand values themselves; obviously we'll need to map the picture cards (_A, K, Q, J and T_) to numeric values first. We can store these as a tuple. I'm putting the creation of the values tuple, as well as the type tuple, in the dataclass `__post_init__` method, declaring the attributes to hold these with `field(init=False)`.


In [1]:
# pyright: strict

import typing as t
from collections import Counter
from dataclasses import dataclass, field
from math import sumprod

Card = t.Literal["A", "K", "Q", "J", "T", "9", "8", "7", "6", "5", "4", "3", "2"]


_card_value: dict[Card, int] = {
    c: i for i, c in enumerate(reversed(t.get_args(Card)), 2)
}


@dataclass
class CamelPokerHand:
    cards: tuple[Card, Card, Card, Card, Card]
    bid: int

    _values: tuple[int, int, int, int, int] = field(init=False)
    _type: tuple[int, ...] = field(init=False)

    def __post_init__(self) -> None:
        self._values = tuple[int, int, int, int, int](
            map(_card_value.__getitem__, self.cards)
        )
        self._type = tuple(v for _, v in Counter(self.cards).most_common())

    def __lt__(self, other: "CamelPokerHand") -> bool:
        return (self._type, self._values) < (other._type, other._values)

    @classmethod
    def from_line(cls, line: str) -> "CamelPokerHand":
        hand, bid = line.split()
        cards = t.cast(tuple[Card, Card, Card, Card, Card], tuple(hand))
        return cls(cards, int(bid))


def total_winnings(*hands: CamelPokerHand) -> int:
    return int(sumprod((h.bid for h in sorted(hands)), range(1, len(hands) + 1)))


test_hands = """\
32T3K 765
T55J5 684
KK677 28
KTJJT 220
QQQJA 483
""".splitlines()
assert total_winnings(*map(CamelPokerHand.from_line, test_hands)) == 6440

In [2]:
import aocd

hand_lines = aocd.get_data(day=7, year=2023).splitlines()
hands = [CamelPokerHand.from_line(line) for line in hand_lines]
print("Part 1:", total_winnings(*hands))

Part 1: 251216224


# Joking around

For part two, using jokers is trivial because we already have our ordered card counts. One of those counts will be for the jokers in the hand. Remove that count from the type tuple and add the joker count to the highest count remaining. Only apply this if there are between 1 and 4 jokers in the hand; with 5 jokers you can't improve on the 5 of a kind hand you already have.


In [3]:
# pyright: strict
_JV = _card_value["J"]


class JokerCamelPokerHand(CamelPokerHand):
    def __post_init__(self) -> None:
        super().__post_init__()
        self._values = tuple[int, int, int, int, int](
            1 if v == _JV else v for v in self._values
        )
        if 0 < (jcount := self._values.count(1)) < 5:
            # add the jokers to the most-common non-joker card count
            type_ = list(self._type)
            type_.remove(jcount)
            type_[0] += jcount
            self._type = tuple(type_)


assert total_winnings(*map(JokerCamelPokerHand.from_line, test_hands)) == 5905

In [4]:
hands = [JokerCamelPokerHand.from_line(line) for line in hand_lines]
print("Part 1:", total_winnings(*hands))

Part 1: 250825971
